# Computational Economics and Nonlinear Modeling (5)

##  Dolo and applications

PSE Summer School, 2017

Pablo Winant

# Quick presentation of Dolo

## What is it ?

- Dynare for nonlinear models but:
    - much more modular
    - based on a complete opensource stack
        - lookup Dyno.jl if interested in Dynare/Julia
    
- Opensource:
    - developped during my thesis
    - then at the IMF
    - then at the BOE
    
- Main contributors:
    - Spencer Lyon (NYU)
    - Anastasia Zhutova (PSE)
    - James Graham (NYU)
    - anybody is welcome to help / report bugs / ask questions

Goals: 

- separate model definitions from the solution routines
- also leverage the programming language for anything that is not the model:
    - graphs
    - loops, ...
- aim for high performance generic algorithms
    - JIT, GPU, ...

## The YAML file

- uses YAML standard
- contains:
    - declarations
    - equations
        - same as Dynare conventions for timing
    - calibration 
    - exogenous shock
    - calibration
    - options

## Examples

github dolo_models

Ongoing documentation

## Test everything is working

- Import the file `rbc.yaml`
- Use the model object to compute steady-state residuals
- Compute linear perturbation solution `perturbate`
- Compute nonlinear solution `time_iteration`
- Compare decision rules (using `tabulate`)

# Add a catastrophic shock to the RBC model

There is an implementation of the RBC model which uses a Markov Chain for exogenous shocks. It is included in the examples as `rbc_dtmscc.yaml`. 


In the orginal implementation, the law of motion for the capital is:

  $$k_t = (1-\delta)k_{t-1} + i_{t-1}$$

The productivity process depends on an AR1:

  $$z_t = \rho z_{t-1} + \epsilon_{z,t}$$

and is given by:

  $$A_t = \exp(z_t)$$
  
The goal of this exercise is to add a catastrophic event to the model. This is done by defining two additional Markov states $x_t$ and $y_t$ defined as follows:

- $x_t$ is equal to 1 in the first period the disaster hits the economy
- $y_t$ is equal to 1 as long as the economy has not recovered (probability $p$ of staying in the disaster state)
  
When there is no disaster ($x_t=y_t=0$) there is a probability $u=3\%$ that a disaster happens next period.

Given the parameters $c_k=30\%$ and  $c_z = 1 - \left( 1- c_k \right)^{1-\alpha}$ the equations are amended as follows:

$$k_t = (1-\delta)k_{t-1}*(1-c_k x_t)+ i_{t-1}$$

$$A_t = \exp(z_t)*(1-c_z y_t)$$

Save `rbc_mc.yaml` as `rbc_disaster.yaml` and add the above modifications. Solve and simulate the new version.

Use `time_iteration`, `tabulate`, `simulate`


# Simple monetary rules and zero lower bound

This part is inspired by the [paper](https://www.ecb.europa.eu/pub/pdf/scpwps/ecbwp377.pdf?374d600f5c7511c6f962e8480cd95a37): *Optimal Monetary Policy Under Commitment with a Zero Bound on Nominal Interest Rages* from Adam and Bili

Objective : replicate the model with simple rules.

The model is defined by the following equations:

$$\pi_{t} = \beta \; \pi_{t+1} + \lambda \; y_{t} + u_{t}$$
$$y_{t} = y_{t+1} - \phi \; \left(i_{t} - \pi_{t+1}\right) + g_{t}$$
$$u_{t} = \rho_{u} \; u_{t-1} + \epsilon_{u,t}$$
$$g_{t} = \rho_{g} \; g_{t-1} + \epsilon_{g,t}$$

We will try to solve the model under the assumption that the nominal interest rate is set by a truncated Taylor Rule: 

$$i_t = \max(0, c_y \; y_t + c_{\pi} \; \pi_t )$$

We will use the `RBC` calibration from the paper:

$  \beta = 0.9913 $, $  r^{\star} =1/\beta-1 $, $  \alpha = 0.007 $, $  \lambda = 0.057 $, $  \phi = 1.00 $,
$  \rho_u = 0.36 $, $  \rho_g = 0.8 $, $  \sigma_u = 0.00171 $, $  \sigma_g = 0.00294 $

As for the Taylor Rule, as a starting point: $ c_{\pi}=1.5$, $c_y = 0.5$

## Question 1: preliminary steps

Answer the following questions:

- what type of exogenous to use ?
- which are the states ? the controls ? the exogenous shocks ? the parameters ?

Which algorithms can be used to solve:

- the non-constrained problem
- the constrained one ?

## Question 2: write the model file

Write in that order:

- symbols
- equations
- calibration
- options:
    - distribution of shocks
    - (approximation grid)

Import and check that everything is OK.

Use: `yaml_import`.

## Question 3: first order solution

- produce a first order approximation
- optional:
    - inspect IRFS
    - simulate and check how often constraint is hit
    - change value of `c_pi` and verify Taylor principle
    
Use: `approximate_controls`, `simulate`, `plot_irfs`, `model.calibration`  and `model.set_calibration`

## Question 4: constrained perfect foresight solution

- reformulate the constraint as a complementarity condition
- compute a perfect foresight solution for an initial shock on $\epsilon_u$ and $\epsilon_g$
    - optional: what happens if shock happens in period $t=2$ rather than $t=0$ ?
    
Use: `deterministic_solve`, `plot_irfs`